# E-commerce Clickstream Data Exploration

This notebook demonstrates exploratory data analysis on the processed clickstream data.

## Contents
1. Setup and Data Loading
2. Basic Statistics
3. User Behavior Analysis
4. Product Performance
5. Conversion Analysis

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Setup and Data Loading

In [ ]:
# Define data paths
data_path = Path('../data/processed')

# Load processed data
daily_metrics = pd.read_parquet(data_path / 'fact_daily_metrics')
session_metrics = pd.read_parquet(data_path / 'fact_sessions')
user_metrics = pd.read_parquet(data_path / 'dim_user_metrics')
product_metrics = pd.read_parquet(data_path / 'dim_product_metrics')

print(f"Loaded {len(daily_metrics)} days of data")
print(f"Loaded {len(session_metrics)} sessions")
print(f"Loaded {len(user_metrics)} users")
print(f"Loaded {len(product_metrics)} products")

## 2. Basic Statistics

In [ ]:
# Overall metrics
print("=" * 60)
print("OVERALL METRICS")
print("=" * 60)
print(f"Total Revenue: ${daily_metrics['total_revenue'].sum():,.2f}")
print(f"Total Purchases: {daily_metrics['total_purchases'].sum():,}")
print(f"Average Daily Revenue: ${daily_metrics['total_revenue'].mean():,.2f}")
print(f"Average Conversion Rate: {daily_metrics['conversion_rate'].mean():.2f}%")
print(f"Average Order Value: ${daily_metrics['avg_order_value'].mean():.2f}")

In [ ]:
# Daily metrics summary
daily_metrics.describe()

## 3. User Behavior Analysis

In [ ]:
# User segmentation
segment_distribution = user_metrics['user_segment'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart
axes[0].pie(segment_distribution.values, labels=segment_distribution.index,
            autopct='%1.1f%%', startangle=90)
axes[0].set_title('User Segment Distribution')

# Bar chart with revenue
segment_revenue = user_metrics.groupby('user_segment')['total_revenue'].sum().sort_values(ascending=False)
axes[1].bar(segment_revenue.index, segment_revenue.values)
axes[1].set_ylabel('Total Revenue ($)')
axes[1].set_title('Revenue by User Segment')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Session duration analysis
session_metrics['session_duration_minutes'] = session_metrics['session_duration_seconds'] / 60

plt.figure(figsize=(12, 6))
plt.hist(session_metrics['session_duration_minutes'][session_metrics['session_duration_minutes'] < 30],
         bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Session Duration (minutes)')
plt.ylabel('Number of Sessions')
plt.title('Session Duration Distribution (< 30 min)')
plt.axvline(session_metrics['session_duration_minutes'].median(), color='red',
            linestyle='--', label=f'Median: {session_metrics["session_duration_minutes"].median():.2f} min')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Product Performance

In [ ]:
# Top 10 products by interactions
top_products = product_metrics.nlargest(10, 'total_interactions')

plt.figure(figsize=(12, 6))
plt.barh(range(len(top_products)), top_products['total_interactions'])
plt.yticks(range(len(top_products)), top_products['product_id'])
plt.xlabel('Total Interactions')
plt.title('Top 10 Products by Interactions')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.show()

In [ ]:
# Category performance
category_performance = product_metrics.groupby('category').agg({
    'total_interactions': 'sum',
    'unique_users': 'sum',
    'num_add_to_cart': 'sum',
    'click_to_cart_rate': 'mean'
}).sort_values('total_interactions', ascending=False)

category_performance

## 5. Conversion Analysis

In [ ]:
# Conversion rate by device
conversion_by_device = session_metrics.groupby('device').agg({
    'session_id': 'count',
    'converted': 'sum'
})
conversion_by_device['conversion_rate'] = (conversion_by_device['converted'] / conversion_by_device['session_id'] * 100)

plt.figure(figsize=(10, 6))
plt.bar(conversion_by_device.index, conversion_by_device['conversion_rate'])
plt.ylabel('Conversion Rate (%)')
plt.xlabel('Device Type')
plt.title('Conversion Rate by Device')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

conversion_by_device

In [ ]:
# Daily trends
daily_metrics_sorted = daily_metrics.sort_values('event_date')

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Users
axes[0, 0].plot(daily_metrics_sorted['event_date'], daily_metrics_sorted['unique_users'], marker='o')
axes[0, 0].set_title('Daily Unique Users')
axes[0, 0].set_ylabel('Users')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Revenue
axes[0, 1].plot(daily_metrics_sorted['event_date'], daily_metrics_sorted['total_revenue'], marker='o', color='green')
axes[0, 1].set_title('Daily Revenue')
axes[0, 1].set_ylabel('Revenue ($)')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Conversion Rate
axes[1, 0].plot(daily_metrics_sorted['event_date'], daily_metrics_sorted['conversion_rate'], marker='o', color='purple')
axes[1, 0].set_title('Daily Conversion Rate')
axes[1, 0].set_ylabel('Conversion Rate (%)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# AOV
axes[1, 1].plot(daily_metrics_sorted['event_date'], daily_metrics_sorted['avg_order_value'], marker='o', color='orange')
axes[1, 1].set_title('Average Order Value')
axes[1, 1].set_ylabel('AOV ($)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary insights
print("\n" + "=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print(f"\n1. Top performing user segment: {user_metrics.groupby('user_segment')['total_revenue'].sum().idxmax()}")
print(f"2. Average session duration: {session_metrics['session_duration_minutes'].mean():.2f} minutes")
print(f"3. Overall conversion rate: {session_metrics['converted'].mean() * 100:.2f}%")
print(f"4. Top category: {product_metrics.groupby('category')['total_interactions'].sum().idxmax()}")
print(f"5. Best converting device: {conversion_by_device['conversion_rate'].idxmax()}")